# Vaccination rate data in the US on county level:

----------

### Full list of US states and counties, their abbreviations and FIPS codes.

`FIPS codes` are essential to merge datasets, as there are many duplicating county names across states, e.g. there are 31 Washington Counties across the US.

---------

### Get latest vaccination rates across the US counties:

   - `CDC covid-19 tracker` [data](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh)
   - `Texas` [data](https://tabexternal.dshs.texas.gov/t/THD/views/COVID-19VaccineinTexasDashboard/Summary?%3Aembed=y)

   - *data extracted on the 4th of October, so last included data were from the `3rd of October`*

------------

### Outcome measure:

Use proportion and `pop over 12` or `pop over 18`, depending on which has more data.

- `pop_full_12plus_per` - % population over 12yo fully vaccinated
- `pop_full_18plus_per` - % population over 18yo fully vaccinated
- `pop_full_65plus_per` - % population over 65yo fully vaccinated


### Final csv `vaccination_rate_US.csv`

- shape `3216,13` - includes `3137 US counties` (except Hawaii) & `79 US territories` (Guam + Puerto Rico)
- columns of interest:

- `fips` code - **`MERGE BY`**!
- `state`
- `county` - try not to merge based on this!
- `state_abb`
- `popfull_per` - not sure if the data for Texas is correct
- `pop_full_12plus_per` - some data missing for 'Alaska' (3%), 'California' (13%), 'Idaho'
- `pop_full_18plus_per` - data not recorded in Texas, California (13%)
- `pop_full_65plus_per`
- `pop_first_12plus_per`  - a lot of missing data on first dose data.
- `pop_first_18plus_per` 
- `pop_first_65plus_per` 
- `svi` - basic sv index that came along with the dataset
- `metro`


------------
## Scraping Wikipedia [US Counties Table](https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county) & [State Abbreviation](https://simple.wikipedia.org/wiki/List_of_U.S._states_by_traditional_abbreviation)

*Using awesome the `Beautiful soup` and tips from [here](https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451)*


*As of 2020, there are 3,142 counties/county equivalents in the US. 
If the 100 county equivalents in the U.S. territories are counted, then the total is 3,242 counties.*

In [1]:
import pandas as pd 
import requests               # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

In [2]:
wikiurl = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county#Table"
table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(wikiurl)
print(response.status_code) # check response

200


In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})

In [4]:
fips = pd.read_html(str(indiatable))
fips = pd.DataFrame(fips[0])
fips.head(n = 5)

,FIPS,County or equivalent,State or equivalent
0,1001,Autauga County,Alabama
1,1003,Baldwin County,Alabama
2,1005,Barbour County,Alabama
3,1007,Bibb County,Alabama
4,1009,Blount County,Alabama


In [5]:
mapping = {'FIPS': 'fips',
           'County or equivalent': 'county',
           'State or equivalent': 'state'}

fips.rename(columns = mapping, inplace = True)

***Clean county/state names***

remove `[*` and `(*`

In [6]:
def clean_county(row):
    row['county'] = row['county'].split('[',1)[0]
    row['state'] = row['state'].split(' (', 1)[0]
    return(row)

fips = fips.apply(clean_county, axis = 'columns')

In [7]:
print(fips.isnull().sum())

print('Number of unique county names in df:', fips['county'].unique().shape[0])
print('Number of counties in df:', fips.shape[0])

fips      0
county    0
state     0
dtype: int64
Number of unique county names in df: 1977
Number of counties in df: 3242


In [8]:
fips['county'].value_counts() 

Washington County        30
Jefferson County         25
Franklin County          24
Lincoln County           23
Jackson County           23
                         ..
Olmsted County            1
Dixon County              1
McKean County             1
Harrisonburg, City of     1
Dolores County            1
Name: county, Length: 1977, dtype: int64

In [9]:
print('unique number of states:', len(fips['state'].unique()))
print()
print(fips['state'].unique())

unique number of states: 57

['Alabama' 'Alaska' 'American Samoa' 'Arizona' 'Arkansas' 'California'
 'Colorado' 'Connecticut' 'Delaware' 'District of Columbia' 'Florida'
 'Georgia' 'Guam' 'Hawaiʻi' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas'
 'Kentucky' 'Louisiana' 'Maine' 'Maryland' 'Massachusetts' 'Michigan'
 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nebraska' 'Nevada'
 'New Hampshire' 'New Jersey' 'New Mexico' 'New York' 'North Carolina'
 'North Dakota' 'Northern Mariana Islands' 'Ohio' 'Oklahoma' 'Oregon'
 'Pennsylvania' 'Puerto Rico' 'Rhode Island' 'South Carolina'
 'South Dakota' 'Tennessee' 'Texas' 'U.S. Minor Outlying Islands' 'Utah'
 'Vermont' 'Virgin Islands' 'Virginia' 'Washington' 'West Virginia'
 'Wisconsin' 'Wyoming']


Fix `'Hawaiʻi'` to `'Hawaii'`


In [10]:
fips.loc[fips['state'] == 'Hawaiʻi', ['state']] = 'Hawaii'
fips[fips['state'] == 'Hawaii']

,fips,county,state
552,15001,Hawaii County,Hawaii
553,15003,"Honolulu, City and County of",Hawaii
554,15005,Kalawao County,Hawaii
555,15007,Kauai County,Hawaii
556,15009,Maui County,Hawaii


In [11]:
fips.to_csv('./data/raw/us_fips_code.csv', index = False)

### `fips` - fips, county, state

*Get abbreviation for the 51 states:*

In [12]:
wikiurl = "https://simple.wikipedia.org/wiki/List_of_U.S._states_by_traditional_abbreviation"
table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(wikiurl)
print(response.status_code)

200


In [13]:
soup = BeautifulSoup(response.text, 'html.parser')
indiatable = soup.find('table',{'class':"wikitable"})
state = pd.read_html(str(indiatable))
state = pd.DataFrame(state[0])
state.head(n = 5)

,State,Traditionalabbreviation,Otherabbreviations
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [14]:
mapping = {'State': 'state',
           'Traditionalabbreviation': 'abb_trad',
           'Otherabbreviations': 'abb'}

state.rename(columns = mapping, inplace = True)

Add `GU` & `PR`  to dictionary

In [15]:
gu = {'state':'Guam', 'abb_trad':'Guam', 'abb': 'GU'}
pr = {'state':'Puerto Rico', 'abb_trad': 'P.R.', 'abb': 'PR'}
#vi = {'state':'Virgin Islands', 'abb_trad':'V.I.', 'abb': 'VI'} # full vacc data available 

for new in (gu, pr):
    state = state.append(new, ignore_index = True)
    
# fix Hawaii, Maine & Georgia:
state.loc[state['state'] == 'Hawaii', ['abb']] = 'HI'
state.loc[state['state'] == 'Maine', ['abb']] = 'ME'
state.loc[state['abb'] == 'GA', ['state']] = 'Georgia'

In [16]:
state_dict = dict(zip(state['state'], state['abb']))
state_dict.keys()

dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'Guam', 'Puerto Rico'])

In [17]:
#state_dict['Georgia'] = state_dict.pop('Template:Country data Georiga (U.S. state)) Georgia')

print(state_dict.keys())
print()
print(len(state_dict.keys()))

dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming', 'Guam', 'Puerto Rico'])

53


In [18]:
state.to_csv('./data/raw/us_states.csv', index = False)

---------------
## Final US `county/state data`

### `us` - fips, county, state

In [19]:
def get_abbr(row):
    
    row['state_abb'] = state_dict.get(row['state'])    
    return(row)
    
us = fips.apply(get_abbr, axis = 'columns')
us.head(n = 5)

,fips,county,state,state_abb
0,1001,Autauga County,Alabama,AL
1,1003,Baldwin County,Alabama,AL
2,1005,Barbour County,Alabama,AL
3,1007,Bibb County,Alabama,AL
4,1009,Blount County,Alabama,AL


In [20]:
print('Unique states', us['state_abb'].unique().shape) # the 55 = None
print('Unique fips', us['fips'].unique().shape)
print('Unique counties', us['county'].unique().shape)

Unique states (54,)
Unique fips (3242,)
Unique counties (1977,)


`Unique states (54,)` - 51 states + 2 territories + None

21 NaN for state_abb - terriroties not on the vacc dataset

In [21]:
us.isnull().sum()

fips          0
county        0
state         0
state_abb    21
dtype: int64

In [22]:
us.to_csv('./data/raw/us_states_county_all.csv', index = False)

In [23]:
us = us.dropna(subset = ['state_abb'])
print(us.shape)

(3221, 4)


`total 3221 counties` - `3142` US + `79` terr

In [24]:
us.to_csv('./data/raw/us_states_county.csv', index = False)

--------------

## CDC Covid-19 tracker `vaccination rate`

*Variables:*

- date - date data reported to the CDC 
- `FIPS` code
- `recip_county`
- `recip_state`
- `series_complete_pop_pct` - Percent of people who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction and county where recipient lives
- sereies_complete_yes - Total number of people who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction and county where recipient lives
- series_complete_12plus - Total number of people 12+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction where recipient lives
- `Series_Complete_12PlusPop_Pct` - Percent of people 12+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction where recipient lives
- Series_Complete_18Plus - Total number of people 18+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction and county where recipient lives
- `Series_Complete_18PlusPop_Pct` - Percent of people 18+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction and county where recipient lives
- Series_Complete_65Plus - Total number of people 65+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction where recipient lives
- `Series_Complete_65PlusPop_Pct` - Percent of people 65+ who are fully vaccinated (have second dose of a two-dose vaccine or one dose of a single-dose vaccine) based on the jurisdiction where recipient lives
- Completeness_pct - Represents the proportion of fully vaccinated people whose Federal Information Processing Standards (FIPS) code is reported and matches a valid county FIPS code in the jurisdiction.  
- `Administered_Dose1_Pop_Pct` - Percent of Total Pop with at least one Dose by State of Residence
- `SVI_CTGY` - SVI index
- `Series_Complete_Pop_Pct_SVI` - Percent of Population Fully Vaccinated/SVI
- `Metro_status` - Metro/Non-metro

*FULL list of variables can be found [here](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh).*



*Notes:*

- no data for `Texas`
- original data 971k rows / 32 columns 
- county number 3223
- data for `Puerto Rico` & `Guam`
- data for `Virgin Islnads` not complete and will be dropped. 
- `Hawaii` no entries for prop data
- No data on the following US territories: Americal Samoa, U.S. Minor Outlying Islands, Northern Mariana Islands

In [25]:
import numpy as np
import matplotlib.pyplot as plt

In [26]:
cdc = pd.read_csv('./data/raw/vaccination_rate_CDC_031021.csv')
print('cdc df shape:', cdc.shape)
print(cdc.columns)

cdc df shape: (3282, 32)
Index(['Date', 'FIPS', 'MMWR_week', 'Recip_County', 'Recip_State',
       'Series_Complete_Pop_Pct', 'Series_Complete_Yes',
       'Series_Complete_12Plus', 'Series_Complete_12PlusPop_Pct',
       'Series_Complete_18Plus', 'Series_Complete_18PlusPop_Pct',
       'Series_Complete_65Plus', 'Series_Complete_65PlusPop_Pct',
       'Completeness_pct', 'Administered_Dose1_Recip',
       'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'SVI_CTGY',
       'Series_Complete_Pop_Pct_SVI', 'Series_Complete_12PlusPop_Pct_SVI',
       'Series_Complete_18PlusPop_Pct_SVI',
       'Series_Complete_65PlusPop_Pct_SVI', 'Metro_status',
       'Series_Complete_Pop_Pct_UR_Equity',
       'Series_Complete_12PlusPop_Pct_UR_Equity',
       'Se

In [27]:
cdc_upd = cdc[['FIPS','Recip_County','Recip_State','Series_Complete_Pop_Pct',
              'Series_Complete_12PlusPop_Pct', 'Series_Complete_18PlusPop_Pct',
              'Series_Complete_65PlusPop_Pct', 'Administered_Dose1_Pop_Pct',
              'Administered_Dose1_Recip_12PlusPop_Pct', 'Administered_Dose1_Recip_18PlusPop_Pct',
              'Administered_Dose1_Recip_65PlusPop_Pct', 'SVI_CTGY',
              'Metro_status']]

mapping = {'FIPS': 'fips',
           'Recip_County': 'county',
           'Recip_State': 'state_abb',
           'Series_Complete_Pop_Pct': 'popfull_per',
           'Series_Complete_12PlusPop_Pct': 'pop_full_12plus_per', 
           'Series_Complete_18PlusPop_Pct': 'pop_full_18plus_per',
           'Series_Complete_65PlusPop_Pct': 'pop_full_65plus_per', 
           'Administered_Dose1_Pop_Pct': 'pop_first_per',
           'Administered_Dose1_Recip_12PlusPop_Pct': 'pop_first_12plus_per', 
           'Administered_Dose1_Recip_18PlusPop_Pct': 'pop_first_18plus_per',
           'Administered_Dose1_Recip_65PlusPop_Pct': 'pop_first_65plus_per', 
           'SVI_CTGY': 'svi',
           'Metro_status': 'metro'}

cdc_upd = cdc_upd.rename(columns = mapping)

In [28]:
cdc_upd.head(n = 5)

,fips,county,state_abb,popfull_per,pop_full_12plus_per,pop_full_18plus_per,pop_full_65plus_per,pop_first_per,pop_first_12plus_per,pop_first_18plus_per,pop_first_65plus_per,svi,metro
0,1001,Autauga County,AL,34.0,39.9,41.8,61.7,43.1,50.6,52.6,74.5,B,Metro
1,1003,Baldwin County,AL,42.7,49.5,51.9,75.3,54.4,63.0,65.7,91.1,A,Metro
2,1005,Barbour County,AL,35.7,41.1,43.0,66.1,46.6,53.8,56.0,76.3,D,Non-metro
3,1007,Bibb County,AL,29.7,34.2,35.7,57.0,37.3,42.8,44.7,64.5,C,Metro
4,1009,Blount County,AL,26.4,31.0,32.9,49.5,32.8,38.5,40.6,57.3,B,Metro


In [29]:
np.round((cdc_upd.isnull().sum() / cdc_upd.shape[0]) * 100, 2)
cdc_upd.isnull().sum() 

fips                     0
county                   0
state_abb                0
popfull_per              0
pop_full_12plus_per     45
pop_full_18plus_per      0
pop_full_65plus_per      0
pop_first_per            0
pop_first_12plus_per    45
pop_first_18plus_per     0
pop_first_65plus_per     0
svi                     63
metro                   63
dtype: int64

In [30]:
cdc_upd['fips'].describe() # 3224 + UNK
# cdc_upd['fips'].count()  2382
# cdc_upd['fips'].unique() 3225

count     3282
unique    3225
top        UNK
freq        58
Name: fips, dtype: object

In [31]:
# cdc_upd[cdc_upd['fips'] == 'UNK']['county']
cdc_upd['fips'] = cdc_upd['fips'].replace('UNK', np.nan)
cdc_upd['county'] = cdc_upd['county'].replace('Unknown County', np.nan)
cdc_upd.dropna(subset = ['fips','county'], inplace = True)

# cdc_upd.info()   # fips object
# us.info()        # fips int64
cdc_upd['fips'] = cdc_upd['fips'].astype(int)

In [32]:
vacc = pd.merge(us[['fips', 'state']], cdc_upd, how = 'left', on = 'fips')
vacc.head(n = 5)

,fips,state,county,state_abb,popfull_per,pop_full_12plus_per,pop_full_18plus_per,pop_full_65plus_per,pop_first_per,pop_first_12plus_per,pop_first_18plus_per,pop_first_65plus_per,svi,metro
0,1001,Alabama,Autauga County,AL,34.0,39.9,41.8,61.7,43.1,50.6,52.6,74.5,B,Metro
1,1003,Alabama,Baldwin County,AL,42.7,49.5,51.9,75.3,54.4,63.0,65.7,91.1,A,Metro
2,1005,Alabama,Barbour County,AL,35.7,41.1,43.0,66.1,46.6,53.8,56.0,76.3,D,Non-metro
3,1007,Alabama,Bibb County,AL,29.7,34.2,35.7,57.0,37.3,42.8,44.7,64.5,C,Metro
4,1009,Alabama,Blount County,AL,26.4,31.0,32.9,49.5,32.8,38.5,40.6,57.3,B,Metro


In [33]:
vacc.isnull().sum()

fips                     0
state                    0
county                   0
state_abb                0
popfull_per              0
pop_full_12plus_per     44
pop_full_18plus_per      0
pop_full_65plus_per      0
pop_first_per            0
pop_first_12plus_per    44
pop_first_18plus_per     0
pop_first_65plus_per     0
svi                      2
metro                    2
dtype: int64

In [34]:
vacc['fips'].unique().shape[0]

3221

In [35]:
vacc.shape

(3221, 14)

----------------

### `Texas data`

*Variables:*

- `County_name` (but no code)
- `Estimated Coverage (12+ 1 Dose)`
- `Estimated Coverage (12+ Fully)` 
- `Estimated Coverage (65+ 1 Dose)`
- `Estimated Coverage (65+ Fully)`
- `Percentage of Population Vaccinated` - unclear how was this derived. 


*Notes:*

- no data on 18+

In [36]:
texas = pd.read_csv('./data/raw/vaccination_rate_texas_031021.csv')
texas.head(n = 5)

,Client County Name,Estimated Coverage (12+ 1 Dose),Estimated Coverage (12+ Fully),Estimated Coverage (65+ 1 Dose),Estimated Coverage (65+ Fully),Percentage of Population Vaccinated,County Name,PHR Critical Pop,Latitude (generated),Longitude (generated),FULLY_VACCINATED_CALC,Population 16Up,TOTAL_PEOPLE_CALC
0,ANDERSON,0.472375,0.405322,0.699007,60.76%,0.472375,Anderson,4/5N,31.7951,-95.6887,20534,48045,23931
1,ANDREWS,0.549889,0.476553,0.868522,79.81%,0.549889,Andrews,09-Oct,32.2885,-102.6379,7083,13503,8173
2,ANGELINA,0.539801,0.469249,0.793857,74.03%,0.539801,Angelina,4/5N,31.2769,-94.6355,34014,67056,39128
3,ARANSAS,0.703849,0.613132,0.900162,81.86%,0.703849,Aransas,11,28.2209,-96.8754,12774,19803,14664
4,ARCHER,0.548322,0.493503,0.785927,74.72%,0.548322,Archer,02-Mar,33.6138,-98.6883,3646,6921,4051


In [37]:
mapping = {'Client County Name': 'county_name', 
        'Estimated Coverage (12+ 1 Dose)': 'pop_first_12plus_per',
        'Estimated Coverage (12+ Fully)': 'pop_full_12plus_per', 
        'Estimated Coverage (65+ 1 Dose)': 'pop_first_65plus_per',
        'Estimated Coverage (65+ Fully)': 'pop_full_65plus_per', 
        'Percentage of Population Vaccinated': 'popfull_per',
        'County Name ': 'county', 
        'PHR Critical Pop' : 'phr_crit_pop', 
        'Latitude (generated)': 'lat',
        'Longitude (generated)': 'long', 
        'FULLY_VACCINATED_CALC' : 'full_vac_num',  
        'Population 16Up': 'pop_16plus',   # old census
        'TOTAL_PEOPLE_CALC': 'total_ppl'}  # old census

texas.rename(columns = mapping, inplace = True)

texas.head(n = 5)

,county_name,pop_first_12plus_per,pop_full_12plus_per,pop_first_65plus_per,pop_full_65plus_per,popfull_per,county,phr_crit_pop,lat,long,full_vac_num,pop_16plus,total_ppl
0,ANDERSON,0.472375,0.405322,0.699007,60.76%,0.472375,Anderson,4/5N,31.7951,-95.6887,20534,48045,23931
1,ANDREWS,0.549889,0.476553,0.868522,79.81%,0.549889,Andrews,09-Oct,32.2885,-102.6379,7083,13503,8173
2,ANGELINA,0.539801,0.469249,0.793857,74.03%,0.539801,Angelina,4/5N,31.2769,-94.6355,34014,67056,39128
3,ARANSAS,0.703849,0.613132,0.900162,81.86%,0.703849,Aransas,11,28.2209,-96.8754,12774,19803,14664
4,ARCHER,0.548322,0.493503,0.785927,74.72%,0.548322,Archer,02-Mar,33.6138,-98.6883,3646,6921,4051


In [38]:
# remove some vars
texas = texas[['county','pop_first_12plus_per','pop_full_12plus_per','pop_first_65plus_per',
               'pop_full_65plus_per', 'popfull_per']]

# fix pop_full_65plus_per:
texas.head(n = 5)

,county,pop_first_12plus_per,pop_full_12plus_per,pop_first_65plus_per,pop_full_65plus_per,popfull_per
0,Anderson,0.472375,0.405322,0.699007,60.76%,0.472375
1,Andrews,0.549889,0.476553,0.868522,79.81%,0.549889
2,Angelina,0.539801,0.469249,0.793857,74.03%,0.539801
3,Aransas,0.703849,0.613132,0.900162,81.86%,0.703849
4,Archer,0.548322,0.493503,0.785927,74.72%,0.548322


In [39]:
texas['pop_full_65plus_per'] = texas['pop_full_65plus_per'].str.replace('%','').astype(float)/100
texas['county'] = texas['county']+' County' 

texas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 6 columns):
county                  254 non-null object
pop_first_12plus_per    254 non-null float64
pop_full_12plus_per     254 non-null float64
pop_first_65plus_per    254 non-null float64
pop_full_65plus_per     254 non-null float64
popfull_per             254 non-null float64
dtypes: float64(5), object(1)
memory usage: 12.0+ KB


In [40]:
us_non_texas_counties = vacc[vacc['state'] != 'Texas']['county'].values.tolist()
print('Total non Texas county number: ', len(us_non_texas_counties))
print('Unique county number: ', len(set(us_non_texas_counties)))

Total non Texas county number:  2967
Unique county number:  1799


In [41]:
us_texas_counties = vacc[vacc['state'] == 'Texas']['county'].values.tolist()
texas_counties = texas['county'].values.tolist()
print(len(us_texas_counties))
print(len(texas_counties))

print('symmetric_difference:', list(set(us_texas_counties).symmetric_difference(set(texas_counties))))
print('Unique Texas counties: ', len(set(texas_counties)))

254
254
symmetric_difference: []
Unique Texas counties:  254


***All Texas counties present in both datasets, but there is a lot of name duplicates, so populate by fips rather than by county name***

-------

In [42]:
texas_fips = us[us['state'] == 'Texas']
texas_fips_list = texas_fips['fips'].tolist()

In [43]:
# texas.describe()
texas = pd.merge(texas, texas_fips, how = 'left', on = 'county')

In [44]:
# test individual counties against map on website:
# test individual counties for fips 
texas[texas['county'] == 'Gaines County']
texas[texas['county'] == 'Webb County']

,county,pop_first_12plus_per,pop_full_12plus_per,pop_first_65plus_per,pop_full_65plus_per,popfull_per,fips,state,state_abb
239,Webb County,0.9999,0.892335,0.988745,0.8788,0.9999,48479,Texas,TX


*Populate the CDC df with Texas data (lazy):*

In [45]:
dict_pop_first_12plus_per = dict(zip(texas['fips'], texas['pop_first_12plus_per']))
dict_pop_full_12plus_per = dict(zip(texas['fips'], texas['pop_full_12plus_per']))
dict_pop_first_65plus_per = dict(zip(texas['fips'], texas['pop_first_65plus_per']))
dict_pop_full_65plus_per = dict(zip(texas['fips'], texas['pop_full_65plus_per']))
dict_pop_full_per = dict(zip(texas['fips'], texas['popfull_per']))


def fill_row(row):
    
    # NOTE MULTIPLY TO GET % DATA:
    if row['fips'] in texas_fips_list:
        row['pop_first_12plus_per'] = dict_pop_first_12plus_per.get(row['fips'])*100    
        row['pop_full_12plus_per'] = dict_pop_full_12plus_per.get(row['fips']) *100    
        row['pop_first_65plus_per'] = dict_pop_first_65plus_per.get(row['fips'])*100 
        row['pop_full_65plus_per'] = dict_pop_full_65plus_per.get(row['fips']) *100
        row['popfull_per'] = dict_pop_full_per.get(row['fips']) *100
    
    return(row)

vacc = vacc.apply(fill_row, axis = 'columns')

In [46]:
vacc.isnull().sum()

fips                     0
state                    0
county                   0
state_abb                0
popfull_per              0
pop_full_12plus_per     44
pop_full_18plus_per      0
pop_full_65plus_per      0
pop_first_per            0
pop_first_12plus_per    44
pop_first_18plus_per     0
pop_first_65plus_per     0
svi                      2
metro                    2
dtype: int64

**many cells are populated with 0s due to lack of data**

In [47]:
vacc.replace(0, np.nan).isnull().sum()

fips                      0
state                     0
county                    0
state_abb                 0
popfull_per              13
pop_full_12plus_per      58
pop_full_18plus_per     267
pop_full_65plus_per      13
pop_first_per           408
pop_first_12plus_per    286
pop_first_18plus_per    495
pop_first_65plus_per    241
svi                       2
metro                     2
dtype: int64

#### Replace `0s` with `np.nan`

In [48]:
vacc.replace(0, np.nan, inplace = True)

In [49]:
var = ['popfull_per', 'pop_full_12plus_per','pop_full_18plus_per','pop_full_65plus_per',
       'pop_first_per','pop_first_12plus_per','pop_first_18plus_per','pop_first_65plus_per']

for v in var:
    vacc[v] = np.round(vacc[v].astype('float'),1)


In [50]:
# check distributions of vaccination data:

# for var in ['popfull_per', 'pop_full_12plus_per','pop_full_18plus_per','pop_full_65plus_per',
#           'pop_first_per','pop_first_12plus_per','pop_first_18plus_per','pop_first_65plus_per']:
    
    # plt.subplots(1, figsize = (4.5,3))
    # plt.title(var)
    # plt.hist(vacc[var])

In [51]:
vacc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3221 entries, 0 to 3220
Data columns (total 14 columns):
fips                    3221 non-null int64
state                   3221 non-null object
county                  3221 non-null object
state_abb               3221 non-null object
popfull_per             3208 non-null float64
pop_full_12plus_per     3163 non-null float64
pop_full_18plus_per     2954 non-null float64
pop_full_65plus_per     3208 non-null float64
pop_first_per           2813 non-null float64
pop_first_12plus_per    2935 non-null float64
pop_first_18plus_per    2726 non-null float64
pop_first_65plus_per    2980 non-null float64
svi                     3219 non-null object
metro                   3219 non-null object
dtypes: float64(8), int64(1), object(5)
memory usage: 377.5+ KB


## observations final vacc data:

In [57]:
for v in var:
    print(v)
    print(set(vacc[vacc[v].isnull()]['state'].values.tolist()))
    print()
    
#vacc[vacc.isnull().any(axis=1)].shape

popfull_per
{'California'}

pop_full_12plus_per
{'Alaska', 'California', 'Idaho'}

pop_full_18plus_per
{'California', 'Texas'}

pop_full_65plus_per
{'California'}

pop_first_per
{'Michigan', 'Nebraska', 'New Mexico', 'California', 'South Dakota', 'Georgia', 'Texas'}

pop_first_12plus_per
{'Alaska', 'Michigan', 'Kentucky', 'Minnesota', 'Nebraska', 'New Mexico', 'California', 'Indiana', 'South Dakota', 'Montana', 'Georgia', 'Pennsylvania', 'Virginia', 'Oklahoma', 'Colorado', 'Guam', 'Idaho'}

pop_first_18plus_per
{'Alaska', 'Michigan', 'Kentucky', 'Minnesota', 'Nebraska', 'New Mexico', 'California', 'South Dakota', 'Montana', 'Georgia', 'Pennsylvania', 'Texas', 'Virginia', 'Oklahoma', 'Colorado', 'Guam', 'Indiana'}

pop_first_65plus_per
{'Alaska', 'Michigan', 'Kentucky', 'Minnesota', 'Nebraska', 'New Mexico', 'California', 'South Dakota', 'Montana', 'Georgia', 'Pennsylvania', 'Virginia', 'Oklahoma', 'Colorado', 'Guam', 'Indiana'}



#### `Missing data` from these states:

- `New Mexico` - 97% data missing for `pop_first_`
- `Hawaii` - all data missing - drop
- `Michigan` - 27% data missing for `pop_first_`
- `Nebraska` - 95% data missing for `pop_first_`
- `South Dakota` - 77% data missing for `pop_first_`
- `California` - 14% data missing for all
- `Texas` - data not recorded for 18over
- `Georgia` - 11% data missing for `pop_first_`
- `Alaska` - very little missing
- `Idaho` - no over 12 data

In [58]:
for s in ('New Mexico', 'Hawaii', 'Michigan', 'Nebraska', 'South Dakota', 'California', 'Texas', 'Georgia',
         'Alaska', 'Idaho'):
    
    print('state: ', s)
    print(vacc[vacc['state'] == s].isnull().sum() / vacc[vacc['state'] == s].shape[0] * 100)
    print()
    
#vacc[vacc['state'] == 'Hawaii'].isnull().sum() / vacc[vacc['state'] == 'Hawaii'].shape[0] * 100

state:  New Mexico
fips                     0.000000
state                    0.000000
county                   0.000000
state_abb                0.000000
popfull_per              0.000000
pop_full_12plus_per      0.000000
pop_full_18plus_per      0.000000
pop_full_65plus_per      0.000000
pop_first_per           48.484848
pop_first_12plus_per    96.969697
pop_first_18plus_per    96.969697
pop_first_65plus_per    96.969697
svi                      3.030303
metro                    0.000000
dtype: float64

state:  Hawaii
fips                   NaN
state                  NaN
county                 NaN
state_abb              NaN
popfull_per            NaN
pop_full_12plus_per    NaN
pop_full_18plus_per    NaN
pop_full_65plus_per    NaN
pop_first_per          NaN
pop_first_12plus_per   NaN
pop_first_18plus_per   NaN
pop_first_65plus_per   NaN
svi                    NaN
metro                  NaN
dtype: float64

state:  Michigan
fips                     0.000000
state                    0.00

**Remove `Hawaii`**

In [63]:
vacc = vacc.drop(vacc[vacc['state'] == 'Hawaii'].index)
vacc = vacc.drop(columns = ['svi','metro'])

**Save data**

In [65]:
vacc.to_csv('./data/clean/vaccination_rate_US.csv', index = False)